In [1]:
# !/home/lishengping/miniconda3/bin/pip install -f https://storage.googleapis.com/jax-releases/libtpu_releases.html jax[tpu]==0.4.35

In [1]:
import os
import sys
import yaml
import json
import base64
from collections import defaultdict
from typing import Tuple
import functools

sys.path.append('/home/lishengping/projects/maxtext/MaxText')
os.environ['HARDWARE'] = 'tpu'

from layers import models
import max_utils
import jax
import orbax
import jax.numpy as jnp
from jax.sharding import Mesh
from flax.traverse_util import flatten_dict, unflatten_dict
from flax import linen as nn
from transformers import AutoTokenizer
from etils import epath
import orbax.checkpoint as ocp

import pyconfig
from jax.sharding import PartitionSpec
from flax.linen import partitioning as nn_partitioning


TOKENIZER_PATH = '/home/lishengping/tokenizer'
if not os.path.exists(TOKENIZER_PATH):
    !gsutil cp -r gs://llm_base_models_us-east5/qwen/tokenizer /home/lishengping/
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_fast=True, trust_remote_code=True)

read_dir = "gs://llm_base_models_europe-west4/v5p_256/7B/PileDCSlimLlama7B32Kx4x256x1v5p_0713/checkpoints"
# read_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/dense_continue_1209/checkpoints'
# read_dir = "gs://llm_base_models_europe-west4/v5p_256/7B/test"

read_dir = epath.Path(read_dir)

config_name = '/home/lishengping/projects/maxtext/MaxText/configs/dc_8x7b_moe.yml'

argv = [None, config_name]
pyconfig.initialize(argv)
config = pyconfig.config
# validate_train_config(config)
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)

2024-12-09 08:58:14.326394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 08:58:14.339980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 08:58:14.340007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 08:58:15.001982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Updating keys from env and command line: []
Running Model: default
Updating keys from model: []
Attempting to initialize the jax distributed system...
Jax distributed system initialized!
System Information: Jax Version: 0.4.35
System Information: Jaxlib Version: 0.4.35
System Information: Jax Backend: PJRT C API
TFRT TPU v6 lite
Built on Oct 21 2024 00:24:02 (1729495442) cl/687888698
Not using emergency checkpoint, ignoring local_checkpoint_directory and local_checkpoint_period
Config param activations_in_float32: False
Config param adam_b1: 0.9
Config param adam_b2: 0.95
Config param adam_eps: 1e-08
Config param adam_eps_root: 0.0
Config param adam_weight_decay: 0.1
Config param allow_split_physical_axes: False
Config param ar_cache_axis_order: 1,2,0,3
Config param async_checkpointing: True
Config param attention: dot_product
Config param autoregressive_decode_assert: 
Config param aux_loss_coef: 0.01
Config param base_emb_dim: 4096
Config param base_mlp_dim: 5632
Config param base_nu

In [6]:
def decode_base64(encoded_str):
    decoded_bytes = base64.b64decode(encoded_str)
    decoded_str = decoded_bytes.decode('utf-8')
    return decoded_str


def mesh_shard_rules(mesh, rules, remove_keys=[]):
    _sharding_dict = {}
    for name, rule in rules.items():
        if isinstance(rule, str):
            rule = json.loads(rule)
        name = decode_base64(name)
        param_key = tuple(name.split('.'))
        remove = any([1 if key in param_key else 0 for key in remove_keys])
        if remove: continue
        prule = [tuple(r) if isinstance(r, list) else r for r in rule['partition_spec'] ]
        spec = jax.sharding.PartitionSpec(*prule)
        _sharding_dict[param_key] = jax.sharding.NamedSharding(mesh, spec)
    return _sharding_dict


def rewrite_bucket_sharding(mesh, old_sharding, save_path):
    cur_machine_sharding = {}
    for k, v in old_sharding.items():
        if isinstance(v, str):
            v = json.loads(v)
        v['shape'] = mesh.device_ids.shape
        cur_machine_sharding[k] = v
    save_path = epath.Path(save_path)
    with save_path.open('w') as f:
        json.dump(cur_machine_sharding, f)
    
# mesh length is 8
_sharding_path = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm3p5_7b_sharding'
# dense
_sharding_path = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm3.5-7b-chat-v6/checkpoints/451600/state/_sharding'
# moe with unshare
_sharding_path = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm3p5_7b_with_perlayer_moe_sharding'
_sharding_path = epath.Path(_sharding_path)

remove_keys = ['opt_state', 'step']
dense = True # 模型是dense 还是moe模型
with _sharding_path.open('r') as f:
    _sharding_rules = json.load(f)
_sharding_dict = mesh_shard_rules(mesh, _sharding_rules, remove_keys=remove_keys)
_sharding_dict = unflatten_dict(_sharding_dict)
restore_args = {}
weight_dtype = jnp.bfloat16
for k, v in flatten_dict(_sharding_dict).items():
    joink = '.'.join(k)
    if dense and 'unshare' in joink: 
        continue
    restore_args[k] =  ocp.ArrayRestoreArgs(restore_type=jax.Array, dtype=weight_dtype, sharding=v)
for k, v in restore_args.items():
    print(k)
restore_args = unflatten_dict(restore_args)

In [14]:
# save
save_step = 0
save_dir = "gs://llm_base_models_europe-west4/v5p_256/7B/dense_continue_lr0_2e-4/checkpoints"
save_dir = epath.Path(save_dir)
item = {
    "state": orbax.checkpoint.Checkpointer(orbax.checkpoint.PyTreeCheckpointHandler(use_ocdbt=False))
}
max_mngr = orbax.checkpoint.CheckpointManager(save_dir, item)
max_mngr.save(save_step, {'state': w})

True

In [10]:
# load
load_step = 440000
ckptr = ocp.Checkpointer(ocp.PyTreeCheckpointHandler())
# restore_args: 结构必须和模型结构一致
w = ckptr.restore(read_dir / load_step / 'state', args=ocp.args.PyTreeRestore(restore_args=restore_args))

I0000 00:00:1733735135.361370   26040 gcs_resource.cc:109] Using default AdmissionQueue with limit 32
I0000 00:00:1733735135.365644   27716 google_auth_provider.cc:180] Running on GCE, using service account 887571727717-compute@developer.gserviceaccount.com


In [1]:
# for k, v in flatten_dict(w).items():
#     print(k)
#     # break